# Homework 8
For this homework, _libsvm_ is needed. The easiest way to install this on _Mac OS X_ is using _Homebrew_ with the following command: `brew install libsvm`.

From this homework: questions 7, 8, 9 and 10 will be answered

In [136]:
from svmutil import *
import pandas as pd
from sklearn.cross_validation import KFold
from collections import Counter
import numpy as np

In [21]:
train = pd.read_table("features.train", sep=" +", header=None, engine='python')
train.columns = ["digit", "intensity", "symmetry"]
test = pd.read_table("features.test", sep=" +", header=None, engine='python')
test.columns = ["digit", "intensity", "symmetry"]

1-vs-5: Only keep data about the digits 1 and 5

In [127]:
filtered = train[(train.digit == 1) | (train.digit == 5)]
filtered["digit"].replace(1,float(-1),inplace=True)
filtered["digit"].replace(5,float(1),inplace=True)
filtered = filtered.reset_index(drop=True)

//anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [103]:
input = filtered[["intensity", "symmetry"]]
x=input.values.tolist()
y=y.values.tolist()

In [105]:
m = svm_train(y, x, '-t 1 -d 2 -v 10')

Cross Validation Accuracy = 99.5516%


In [195]:
possible_C = [0.0001, 0.001, 0.01, 0.1, 1]
best_run_C = []
C_val_errors = {c: [] for c in possible_C}
for run in range(100):
    kf = KFold(len(filtered), n_folds=10, shuffle=True)
    C_errors = []
    for C in possible_C:
        fold_errors = []
        for train_index, val_index in kf:
            train_fold = filtered.ix[train_index]
            train_x = train_fold[["intensity", "symmetry"]].values.tolist()
            train_y = train_fold["digit"].values.tolist()
            val_fold = filtered.ix[val_index]
            val_x = val_fold[["intensity", "symmetry"]].values.tolist()
            val_y = val_fold["digit"].values.tolist()
            m = svm_train(train_y, train_x, '-q -t 1 -d 2 -c {}'.format(C))
            p_label, p_acc, p_val = svm_predict(val_y, val_x, m, "-q")
            fold_errors.append(100-p_acc[0])
        mean_fold_error = np.mean(fold_errors)
        C_val_errors[C].append(mean_fold_error)
        C_errors.append(mean_fold_error)
    best_run_C.append(possible_C[np.argmin(C_errors)])


## Question 7

In [196]:
best_c = Counter(best_run_C).most_common(1)[0][0]
print("Best C: ", best_c)

Best C:  0.01


## Question 8

In [197]:
print("Cross validation error when using C=", best_c, ": ", np.mean(C_val_errors[best_c]))

Cross validation error when using C= 0.01 :  0.449109913441
